# Project: **Wrangle OpenStreetMap Data**


### 1. Map Area

* [Campina Grande, PB, Brazil](https://en.wikipedia.org/wiki/Campina_Grande)

This is where my parents live. I’m very interested to see what database querying reveals. I also wish to contribute to improve OpenStreetMap.org making this data chunck more useful for everyone.

### 2. Getting Data

To get the data used in this task:

1. OpenStreetMap: https://www.openstreetmap.org/relation/301079

2. Overpass API: http://overpass-api.de/query_form.html
    
3. Query Form: *(Using the same values given on the EXPORT page in OpenStreetMap)*

    ```ruby
    (
    node(-7.4775,-36.2357,-7.0641,-7.0641);
    <;
    );out meta;

    ```
    
4. Transform OSM in CSV files using script python
5. Insert csv files in a relational database ()


### 3. Inserting Data on Relational Database

1. [Creating *nodes* and *ways* schema](file)


```sql
create table nodes(
	id bigserial,
	lat numeric,
	lon numeric,
	user_name varchar(200),
	uid integer,
	version integer,
	changeset integer,
	timestamp timestamp,

	primary key(id)
);

create table nodes_tags(
	id bigserial,
	key varchar(200),
	type varchar(200),
	value varchar(200)
);

create table ways(
	id bigserial,
	user_name varchar(200),
	uid integer, 
	version integer,
	changeset integer, 
	timestamp timestamp
);

create table ways_tags(
	id bigserial, 
	key varchar(200),
	value varchar(2000), 
	type varchar(200)
);
```

2. Insert values:

```sql
COPY nodes(id,lat,lon,user_name,uid,version,changeset,timestamp) 
FROM '../csvs/nodes.csv' DELIMITER ',' CSV HEADER;

COPY nodes_tags(key,type,id,value) 
FROM '../csvs/nodes_tags.csv' DELIMITER ',' CSV HEADER;

COPY ways(id,user_name,uid,version,changeset,timestamp) 
FROM '../csvs/ways.csv' DELIMITER ',' CSV HEADER;

COPY ways_nodes(id,node_id,position) 
FROM '../csvs/ways_nodes.csv' DELIMITER ',' CSV HEADER;

COPY ways_tags(id,key,value,type) 
FROM '../csvs/ways_tags.csv' DELIMITER ',' CSV HEADER;
```

### Problems Encountered in the Map


### Problem match

### Overabbreviated Street Names


### Postal Codes

# Sort cities by count, descending


# Data Overview and Additional Ideas


### File sizes

### Number of nodes

### Number of ways

### Number of unique users

### Top 10 contributing users


### Number of users appearing only once (having 1 post)


# Additional Ideas


### Contributor statistics and gamification suggestion


### Additional Data Exploration


### Top 10 appearing amenities


### Biggest religion (no surprise here)

### Most popular cuisines

# Conclusion
